# torch.utils.data.SubsetRandomSampler
它是 PyTorch 数据加载 (DataLoader) 里常用的采样器。

## 1. 作用

`SubsetRandomSampler` 的作用是：\
👉 在给定的索引集合中，`随机抽取样本`。

也就是说，它不会打乱整个数据集，而是只在 `指定的子集` 中`随机采样`。
- 常见用途：

    - 划分训练集 / 验证集（比如从一个大数据集里选 80% 训练，20% 验证）
    
    - 在部分数据上调试模型
    
    - 分布式采样时指定样本范围

## 2. 定义方式

In [ ]:
torch.utils.data.SubsetRandomSampler(
    indices,    # 子集的索引列表/张量
    generator=None  # 随机数生成器 (可选，控制随机性)
)


### 参数说明：

- `indices`

    - 必须是一个 索引序列（list / numpy array / torch.Tensor）。

例如 [0, 1, 5, 10] 表示只从这些位置采样。

- `generator`

    - 控制随机性的生成器。

比如 `torch.Generator().manual_seed(42)` 可以保证结果可复现。

---
## 3. 使用示例
示例 1：划分训练集和验证集

In [ ]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms

# 数据准备
transform = transforms.ToTensor()
dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)

# 划分索引
num_samples = len(dataset)  # 60000
indices = list(range(num_samples))
split = int(0.8 * num_samples)  # 80% 训练
train_idx, val_idx = indices[:split], indices[split:]

# 定义采样器
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

# DataLoader 使用采样器
train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)

print(len(train_loader), len(val_loader))  # 750, 188


👉 注意： 当 `sampler` 被指定时，`shuffle=False`（因为采样器本身就决定了顺序）。

---
示例 2：只取一部分数据做调试

In [ ]:
subset_indices = list(range(1000))  # 只用前 1000 个样本
subset_sampler = SubsetRandomSampler(subset_indices)

debug_loader = DataLoader(dataset, batch_size=32, sampler=subset_sampler)

for images, labels in debug_loader:
    print(images.shape)  # torch.Size([32, 1, 28, 28])
    break


In [ ]:
示例 3：固定随机数生成器（保证复现）

In [ ]:
g = torch.Generator().manual_seed(42)
subset_sampler = SubsetRandomSampler(indices, generator=g)


## 4. 小结

`SubsetRandomSampler`：在 `给定索引集合` 中`随机采样`。

- 常用场景：

    - 划分训练/验证集
    
    - 仅使用部分数据调试
    
    - 精确控制采样范围

和 `RandomSampler` 的区别：

- `RandomSampler` 会在`整个数据集`里`随机采样`。

- `SubsetRandomSampler` 只在`指定子集`里`随机采样`。

---
其他`随机采样`:
- 全量随机 → RandomSampler
- 子集随机 → SubsetRandomSampler
- 顺序采样 → SequentialSampler